# Data

In [26]:
import numpy as np
import pandas as pd
import requests
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [34]:
data_folder = 'data/'

In [35]:
def clean_csv(dataframe, indicator):
    df = dataframe.copy()
    df.drop(columns=['Indicator Name', 'Indicator Code', 'Country Name'], inplace=True)
    df.dropna(axis=1, inplace=True, how='all')
    df = df.T
    df.columns = df.iloc[0].values
    df.drop(axis=0, index='Country Code', inplace=True)
    #df['WORLD'] = df.sum(axis=1)
    df.reset_index(inplace=True)
    df.rename(columns={'index':'year'}, inplace=True)
    df = df.melt(id_vars=['year'], value_vars=df.drop(columns='year').columns, var_name='country', value_name=indicator)
    df.year = df.year.astype(int)
    return df

In [36]:
df1 = pd.read_csv(data_folder+"CO2.csv", skiprows=4)
df2 = pd.read_csv(data_folder+"electric_power_consumption.csv", skiprows=4)
df3 = pd.read_csv(data_folder+"CO2_2.csv", skiprows=4)

co2 = clean_csv(df1, indicator='co2_per_capita')
electric_power_consumption = clean_csv(df2, indicator='electric_power_consumption')
co2_2 = clean_csv(df3, indicator='co2_total')

In [37]:
df = co2.copy()
corres_country = df1[['Country Name', 'Country Code']]

df['electric_power_consumption'] = electric_power_consumption.electric_power_consumption
df['co2_total'] = co2_2.co2_total

df = pd.merge(df, corres_country, left_on='country', right_on='Country Code')
df.drop(columns='country', inplace=True)
df.rename(columns={'Country Name':'country_name', 'Country Code':'country_code'}, inplace=True)

In [38]:
df

,year,co2_per_capita,electric_power_consumption,co2_total,country_name,country_code
0,1960,NaN,NaN,NaN,Aruba,ABW
1,1961,NaN,NaN,NaN,Aruba,ABW
2,1962,NaN,NaN,NaN,Aruba,ABW
3,1963,NaN,NaN,NaN,Aruba,ABW
4,1964,NaN,NaN,NaN,Aruba,ABW
...,...,...,...,...,...,...
14515,2010,0.612528,606.644,7777.71,Zimbabwe,ZWE
14516,2011,0.741686,636.249,9563.54,Zimbabwe,ZWE
14517,2012,0.594152,608.763,7792.38,Zimbabwe,ZWE
14518,2013,0.874563,628.747,11675.7,Zimbabwe,ZWE


# Dash

In [39]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output


In [40]:
# Build App
app = JupyterDash(__name__)

#fig1 = px.line(df, x='year', y='co2', color='country', hover_name="country", title='CO2 emissions by country')
#fig2 = px.scatter(df, x='electric_power_consumption', y='co2', animation_frame='year', color='country')

country_options = [
    {"label": country_name, "value": country_name}
    for country_name in df.country_name.unique()
]

year_options = [
    {"label": year, "value": year}
    for year in df.year.unique()
]

def filter_dataframe(df, selected_countries, selected_years):
    
    if type(selected_countries)==str:
        selected_countries = [selected_countries]
    elif not selected_countries:
        selected_countries = df["country_name"].unique()
        
    dff = df[
        df["country_name"].isin(selected_countries)
        & (df["year"] > selected_years[0])
        & (df["year"] < selected_years[1])
    ]
    return dff

app.layout = html.Div(
    [
        dcc.Store(id="aggregate_data"),
        # empty Div to trigger javascript file for graph resizing
        html.Div(id="output-clientside"),
        html.Div(
            [
                html.Div(
                    [
                        html.Img(
                            src=app.get_asset_url("CO2.png"),
                            id="CO2-image",
                            style={
                                "height": "60px",
                                "width": "auto",
                                "margin-bottom": "25px",
                            },
                        )
                    ],
                    className="one-third column",
                ),
                html.Div(
                    [
                        html.Div(
                            [
                                html.H3(
                                    "CO2 emissions in the world",
                                    style={"margin-bottom": "0px"},
                                ),
                                html.H5(
                                    "A recent overview", style={"margin-top": "0px"}
                                ),
                            ]
                        )
                    ],
                    className="one-half column",
                    id="title",
                ),
                html.Div(
                    [
                        html.A(
                            html.Button("Learn More", id="learn-more-button"),
                            href="https://leclimatchange.fr/",
                        )
                    ],
                    className="one-third column",
                    id="button",
                ),
            ],
            id="header",
            className="row flex-display",
            style={"margin-bottom": "25px"},
        ),
        html.Div(
            [
                html.Div(
                    [
                        html.P(
                            "Filter by date (or select range in histogram):",
                            className="control_label",
                        ),
                        dcc.RangeSlider(
                            id="year_slider",
                            min=1960,
                            max=2014,
                            value=[1960, 2014],
                            className="dcc_control",
                        ),
                        html.P("Filter by country:", className="control_label"),
                        dcc.Dropdown(
                            id="country_selector",
                            options=country_options,
                            value=[],
                            multi=True,
                            className="dcc_control",
                        ),
                        html.P("Choose a metric:", className="control_label"),
                        dcc.RadioItems(
                            id="metric_selector",
                            options=[
                                {"label": "Metric tons per capita ", "value": "co2_per_capita"},
                                {"label": "Kilo tons ", "value": "co2_total"},
                            ],
                            value="co2_per_capita",
                            labelStyle={"display": "inline-block"},
                            className="dcc_control",
                        )
                    ],
                    className="pretty_container four columns",
                    id="cross-filter-options",
                ),
                html.Div(
                    [
                        html.Div(
                            [
                                html.Div(
                                    [html.H6(id="co2_total_text"), html.P("Mean of CO2 emissions (kt)")],
                                    id="co2_total",
                                    className="mini_container",
                                ),
                                html.Div(
                                    [html.H6(id="co2_per_capita_text"), html.P("Mean of CO2 emissions (metric tons per capita)")],
                                    id="co2_per_capita",
                                    className="mini_container",
                                )
                            ],
                            id="info-container",
                            className="row container-display",
                        ),
                        html.Div(
                            [dcc.Graph(id="graph1")],
                            id="countGraphContainer1",
                            className="pretty_container",
                        ),
                        html.Div(
                            [dcc.Graph(id="graph2")],
                            id="countGraphContainer2",
                            className="pretty_container",
                        ),
                    ],
                    id="right-column",
                    className="eight columns",
                ),
            ],
            className="row flex-display",
        )
    ],
    id="mainContainer",
    style={"display": "flex", "flex-direction": "column"},
)

# Define callback to update graph

@app.callback(
    Output('graph1', 'figure'),
    [Input('country_selector', 'value'),
    Input('year_slider', 'value'),
    Input('metric_selector', 'value')]
)
def update_graph1(selected_countries, selected_years, selected_metric):
    
    dff = filter_dataframe(df, selected_countries, selected_years)
    
    fig = px.line(dff, x='year', y=selected_metric, color='country_name', 
                  title='CO2 emissions')
    fig.update_xaxes(rangeslider_visible=True)
    
    return fig

@app.callback(
    [
        Output("co2_total_text", "children"),
        Output("co2_per_capita_text", "children"),
    ],
    [Input('country_selector', 'value'),
    Input('year_slider', 'value')],
)
def update_text(selected_countries, selected_years):
    
    dff = filter_dataframe(df, selected_countries, selected_years)
    
    return round(dff.co2_total.mean(),2) , round(dff.co2_per_capita.mean(),2) 

@app.callback(
    Output('graph2', 'figure'),
    [Input('country_selector', 'value'),
    Input('year_slider', 'value'),
    Input('metric_selector', 'value')]
)
def update_graph2(selected_countries, selected_years, selected_metric):
    
    dff = filter_dataframe(df, selected_countries, selected_years)
    
    fig = px.choropleth(dff, locations="country_code", color=selected_metric, hover_name="country_name", 
                        animation_frame="year", range_color=[0,20], title='CO2 emissions', color_continuous_scale=px.colors.diverging.RdYlGn[::-1])
    
    return fig

In [42]:
# Run app and display result inline in the notebook
app.run_server(mode='external', port=9000, host='127.0.0.1')

Dash app running on http://127.0.0.1:9000/


In [50]:
fig.show()